Вероятности классов

Условные вероятности

Приложения

Загрузка данных

In [20]:
import pandas as pd

df = pd.read_table('SMSSpamCollection',
                   sep='\t', 
                   header=None,
                   names=['label', 'message'])
 

In [2]:
df

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


Предварительная обработка

In [21]:
df['label'] = df.label.map({'ham': 0, 'spam': 1})
df['label']

0       0
1       0
2       1
3       0
4       0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Name: label, Length: 5572, dtype: int64

In [22]:
#Во-вторых, преобразуйте все символы в сообщении в нижний регистр:
df['message'] = df.message.map(lambda x: x.lower())
df['message']

0       go until jurong point, crazy.. available only ...
1                           ok lar... joking wif u oni...
2       free entry in 2 a wkly comp to win fa cup fina...
3       u dun say so early hor... u c already then say...
4       nah i don't think he goes to usf, he lives aro...
                              ...                        
5567    this is the 2nd time we have tried 2 contact u...
5568                 will ü b going to esplanade fr home?
5569    pity, * was in mood for that. so...any other s...
5570    the guy did some bitching but i acted like i'd...
5571                           rofl. its true to its name
Name: message, Length: 5572, dtype: object

In [23]:
# В-третьих, удалите все знаки препинания:
df['message'] = df.message.str.replace('[^\w\s]', '')
df['message']

0       go until jurong point crazy available only in ...
1                                 ok lar joking wif u oni
2       free entry in 2 a wkly comp to win fa cup fina...
3             u dun say so early hor u c already then say
4       nah i dont think he goes to usf he lives aroun...
                              ...                        
5567    this is the 2nd time we have tried 2 contact u...
5568                  will ü b going to esplanade fr home
5569    pity  was in mood for that soany other suggest...
5570    the guy did some bitching but i acted like id ...
5571                            rofl its true to its name
Name: message, Length: 5572, dtype: object

In [24]:
#df['message'] = df.message.str.replace('[^\w\s]', '')
df['message'][3]

'u dun say so early hor u c already then say'

In [16]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
df['message'] = df['message'].apply(nltk.word_tokenize)

In [26]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
 
df['message'] = df['message'].apply(lambda x: [stemmer.stem(y) for y in x])


In [27]:
from sklearn.feature_extraction.text import CountVectorizer

# Конвертируем список слов в строку слов разделенных пробелами
df['message'] = df['message'].apply(lambda x: ' '.join(x))

count_vect = CountVectorizer()
counts = count_vect.fit_transform(df['message'])


In [30]:
counts

<5572x8169 sparse matrix of type '<class 'numpy.float64'>'
	with 72500 stored elements in Compressed Sparse Row format>

In [28]:
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer().fit(counts)

counts = transformer.transform(counts)

In [29]:
df['message']

0       go until jurong point crazi avail onli in bugi...
1                                   ok lar joke wif u oni
2       free entri in 2 a wkli comp to win fa cup fina...
3             u dun say so earli hor u c alreadi then say
4       nah i dont think he goe to usf he live around ...
                              ...                        
5567    thi is the 2nd time we have tri 2 contact u u ...
5568                      will ü b go to esplanad fr home
5569         piti wa in mood for that soani other suggest
5570    the guy did some bitch but i act like id be in...
5571                              rofl it true to it name
Name: message, Length: 5572, dtype: object

Обучение модели

Теперь, когда мы выполнили извлечение признаков из наших данных, пришло время построить нашу модель. Мы начнем с разделения ваших данных на обучающие и тестовые наборы:

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(counts, df['label'], test_size=0.1, random_state=69)


In [33]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB().fit(X_train, y_train)

Оценка модели

In [41]:
import numpy as np

predicted = model.predict(X_test)
accuracy = np.mean(predicted == y_test)

# numb_spam = np.mean(df['message'] == 1)

print('верность = {0:.1%}'.format(accuracy))
#print('доля спама = {0:.1%}'.format(numb_spam))

верность = 94.8%
доля спама = 0.0%


In [42]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, predicted))


[[482   0]
 [ 29  47]]
